# Análisis de Rendimiento y Tendencias de Ventas

## Tendencias Temporales: Analizar el crecimiento o decrecimiento de las ventas (sale_dollars) por día, mes o año.

### Por día

In [0]:
%sql
-- Calculo de la diferencia (crecimiento/decrecimiento) usando LAG por día
SELECT
  date,
  total_sales,
  previous_day_sales,
  daily_growth_or_decrease
FROM (
  SELECT
    date,
    total_sales,
    -- Obtiene el valor de 'total_sales' de la fila anterior (día anterior)
    LAG(total_sales, 1, 0) OVER (ORDER BY date) AS previous_day_sales,
     -- Calcula la diferencia
    ROUND(total_sales - LAG(total_sales, 1, 0) OVER (ORDER BY date), 2) AS daily_growth_or_decrease
  FROM (
    -- Obtener el total de ventas por día
    SELECT
      d.date,
      ROUND(SUM(f.sale_dollars), 2) AS total_sales
    FROM
      sales.sales_gold.fact_sales AS f
    INNER JOIN
      sales.sales_gold.dim_time AS d ON f.idDate = d.dateId
    GROUP BY
      d.date
  ) AS daily_sales
) AS analysis_with_lag

-- AGREGA EL FILTRO AQUÍ --
--WHERE date = '2021-09-23'  -- Reemplaza con la fecha que deseas analizar

ORDER BY
  date DESC;

Esta consulta devuelve las ventas totales del día actual, ventas totales del día anterior y aumento/disminución del volumen de ventas del día actual con respecto al día anterior, ordenado por día. De querer analizar un día específico se debe introducir una fecha válida en la claúsula WHERE comentada.

### Por mes

In [0]:
%sql
-- Aplicar el filtro de mes/año
SELECT *
FROM (
  -- Calcular el LAG sobre TODOS los meses
  SELECT
    monthly_sales.year,
    monthly_sales.month,
    monthly_sales.month_name,
    monthly_sales.total_sales,
    
    -- Obtiene el valor de 'total_sales' de la fila anterior (ordenada por fecha)
    LAG(total_sales, 1, 0) OVER (ORDER BY year, month) AS previous_month_sales,
    
    -- Calcula la diferencia
    ROUND(total_sales - LAG(total_sales, 1, 0) OVER (ORDER BY year, month), 2) AS monthly_growth_or_decrease
    
  FROM (
    -- Obtener el total de ventas por mes
    SELECT
      d.year,
      d.month,
      d.month_name,
      round(SUM(f.sale_dollars), 2) AS total_sales
    FROM
      sales.sales_gold.fact_sales AS f
    INNER JOIN
      sales.sales_gold.dim_time AS d ON f.idDate = d.dateId
    GROUP BY
      d.year,
      d.month,
      d.month_name
  ) AS monthly_sales

) AS analysis_with_lag -- Esta es la tabla con TODOS los meses y sus LAGs calculados

--WHERE analysis_with_lag.year = 2020 AND analysis_with_lag.month = 8  -- Reemplaza con el año y mes (número)

ORDER BY
  analysis_with_lag.year DESC,
  analysis_with_lag.month DESC;

Esta consulta devuelve las ventas totales de un mes en un año determinado, ventas totales del mes anterior y aumento/disminución del volumen de ventas del mes actual con respecto al mes anterior, ordenados por año-mes. De querer analizar un mes de un año específico se debe introducir un año y un mes(valor númerico) en la claúsula WHERE comentada.

### Por año

In [0]:
%sql
-- Aplicar el filtro de año
SELECT *
FROM (

  -- Calcular el LAG sobre TODOS los años (tu query original)
  SELECT
    year,
    total_sales,
    
    -- Obtiene el valor de 'total_sales' de la fila anterior (año anterior)
    LAG(total_sales, 1, 0) OVER (ORDER BY year) AS previous_year_sales,
    
    -- Calcula la diferencia
    ROUND(total_sales - LAG(total_sales, 1, 0) OVER (ORDER BY year), 2) AS yearly_growth_or_decrease
    
  FROM (
    -- Obtener el total de ventas por año
    SELECT
      d.year,
      ROUND(SUM(f.sale_dollars), 2) AS total_sales
    FROM
      sales.sales_gold.fact_sales AS f
    INNER JOIN
      sales.sales_gold.dim_time AS d ON f.idDate = d.dateId
    GROUP BY
      d.year
  ) AS yearly_sales

) AS analysis_with_lag -- Esta es la tabla con TODOS los años y sus LAGs calculados

-- Filtro por año en específico
--WHERE analysis_with_lag.year = 2021  -- Reemplaza con el año que deseas analizar

ORDER BY
  year DESC;

Esta consulta devuelve las ventas totales de año determinado, ventas totales del año anterior y aumento/disminución del volumen de ventas del año actual con respecto al año anterior, ordenados por año. De querer analizar un año específico se debe introducir el año en la claúsula WHERE comentada.

## Ranking de Puntos de Venta: Determinar las tiendas y condados con mayores volúmenes de venta.

### Tiendas con mayores volúmenes de venta

In [0]:
%sql
SELECT
  s.store_number,
  s.store_name,
  ROUND(SUM(f.sale_dollars), 2) AS total_sales
FROM
  sales.sales_gold.fact_sales AS f
INNER JOIN
  sales.sales_gold.dim_store AS s ON f.idStore = s.idStore
GROUP BY
  s.store_number,
  s.store_name
ORDER BY
  total_sales DESC
LIMIT 10;

En esta consulta se puede visualizar el número de tienda, nombre y total de ventas de las 10 tiendas con mayor volumen de venta histórico, ordenados por volumen de ventas en forma descendente.
De querer agrandar o disminuir la lista se debe manipular el valor de LIMIT.

### Condados con mayores volúmenes de venta

In [0]:
%sql
SELECT
  l.county,
  ROUND(SUM(f.sale_dollars), 2) AS total_sales
FROM
  sales.sales_gold.fact_sales AS f
INNER JOIN
  sales.sales_gold.dim_store AS s ON f.idStore = s.idStore
INNER JOIN
  sales.sales_gold.dim_location AS l ON s.idLocation = l.idLocation
GROUP BY
  l.county
ORDER BY
  total_sales DESC
LIMIT 10;

En esta consulta se puede visualizar el nombre del condado y total de ventas de los 10 condados con mayor volumen de venta histórico, ordenados por volumen de ventas en forma descendente.
De querer agrandar o disminuir la lista se debe manipular el valor de LIMIT.

## Comportamiento de Categorías: Identificar las categorías de productos (category_name) más vendidas y las de mayor crecimiento.

### Categorías más vendidas

In [0]:
%sql
SELECT
  p.category,
  ROUND(SUM(f.sale_dollars), 2) AS total_sales
FROM
  sales.sales_gold.fact_sales AS f
INNER JOIN
  sales.sales_gold.dim_product AS p ON f.idProduct = p.idProduct
GROUP BY
  p.category
ORDER BY
  total_sales DESC
LIMIT 10;

En esta consulta se puede visualizar el nombre de categoría y total de ventas de las 10 categorías con mayor volumen de venta histórico, ordenados por volumen de ventas en forma descendente.
De querer agrandar o disminuir la lista se debe manipular el valor de LIMIT.

### Categorías de mayor crecimiento (interanual)

In [0]:
%sql
-- Calcular el crecimiento y ordenar
SELECT
  year,
  category,
  yearly_sales,
  previous_year_sales,
  -- Crecimiento absoluto en dólares
  ROUND(yearly_sales - previous_year_sales, 2) AS absolute_growth,
  
  -- Crecimiento porcentual (evitando división por cero)
  CASE
    WHEN previous_year_sales > 0 
    THEN ROUND((yearly_sales - previous_year_sales) / previous_year_sales * 100, 2)
    ELSE NULL
  END AS percentage_growth_pct
  
FROM (
  --  Usar LAG para obtener las ventas del año anterior para CADA categoría
  SELECT
    year,
    category,
    yearly_sales,
    LAG(yearly_sales, 1, 0) OVER (PARTITION BY category ORDER BY year) AS previous_year_sales
  FROM (
    --  Obtener el total de ventas por categoría y año
    SELECT
      d.year,
      p.category,
      ROUND(SUM(f.sale_dollars), 2) AS yearly_sales
    FROM
      sales.sales_gold.fact_sales AS f
    INNER JOIN
      sales.sales_gold.dim_product AS p ON f.idProduct = p.idProduct
    INNER JOIN
      sales.sales_gold.dim_time AS d ON f.idDate = d.dateId
    GROUP BY
      d.year,
      p.category
  ) AS sales_by_year
) AS sales_with_lag

-- Ordenar para ver el mayor crecimiento en el año más reciente
ORDER BY
  percentage_growth_pct DESC, absolute_growth DESC
LIMIT 10;

En esta consulta se puede visualizar el desempeño en volumen de ventas de una determinada categoría en un determinado año con respecto al año anterior, en conjunto de metricas como crecimiento absoluto y crecimiento porcentual con respecto al desempeño del año anterior.
Por lo tanto, se visualizan las 10 categorías que tuvieron un mayor crecimiento de un año a otro. Ordenadas por crecimiento porcentual en conjunto con el crecimiento absoluto.

# Análisis de Rentabilidad (Gross Margin)

## Cálculo de Margen

### Margen de beneficio bruto por artículo

In [0]:
%sql
SELECT
  p.itemno,
  p.itemName,
  ROUND(AVG(f.state_bottle_retail), 2) AS avg_retail_price,
  ROUND(AVG(f.state_bottle_cost), 2) AS avg_cost,
  -- Margen en dólares (por botella)
  ROUND(AVG(f.state_bottle_retail) - AVG(f.state_bottle_cost), 2) AS avg_margin_dollars_per_bottle,
  -- Margen porcentual (por botella)
  ROUND(((AVG(f.state_bottle_retail) - AVG(f.state_bottle_cost)) / AVG(f.state_bottle_retail)) * 100, 2) AS avg_margin_pct_per_bottle

FROM
  sales.sales_gold.fact_sales AS f
INNER JOIN
  sales.sales_gold.dim_product AS p ON f.idProduct = p.idProduct
GROUP BY
  p.itemno,
  p.itemName
ORDER BY
  avg_margin_dollars_per_bottle DESC;

En esta consulta se puede visualizar el itemno y nombre de cada producto, junto con su precio de venta promedio y costo promedio.
Adicionalmente, calcula el margen de beneficio promedio por cada botella vendida, tanto en dólares como en porcentaje.
Los resultados están ordenados por el margen en dólares en forma descendente, mostrando los productos más rentables (en dólares por botella) primero.

### Margen de beneficio bruto por categoría

In [0]:
%sql
SELECT
  p.category,
  ROUND(SUM(f.sale_dollars), 2) AS total_revenue,
  ROUND(SUM(f.total_cost), 2) AS total_cost,
  
  -- Margen bruto total en dólares
  ROUND(SUM(f.sale_dollars) - SUM(f.total_cost), 2) AS total_gross_margin_dollars,
  
  -- Margen bruto porcentual total
  ROUND(((SUM(f.sale_dollars) - SUM(f.total_cost)) / SUM(f.sale_dollars)) * 100, 2) AS total_gross_margin_percent

FROM
  sales.sales_gold.fact_sales AS f
JOIN
  sales.sales_gold.dim_product AS p ON f.idProduct = p.idProduct
GROUP BY
  p.category
ORDER BY
  total_gross_margin_dollars DESC;

En esta consulta se visualiza la rentabilidad de cada categoría de producto.
La consulta calcula los ingresos totales, el costo total y el margen de beneficio bruto total para cada categoría, mostrando el resultado tanto en dólares como en porcentaje.
Los resultados están ordenados por el margen en dólares en forma descendente, mostrando las categorías más rentables primero.

### Margen de beneficio bruto por vendedor

In [0]:
%sql
SELECT
  s.vendor_no,
  ROUND(SUM(f.sale_dollars), 2) AS total_revenue,
  ROUND(SUM(f.total_cost), 2) AS total_cost,
  
  -- Margen bruto total en dólares
  ROUND(SUM(f.sale_dollars) - SUM(f.total_cost), 2) AS total_gross_margin_dollars,
  
  -- Margen bruto porcentual total
  ROUND(((SUM(f.sale_dollars) - SUM(f.total_cost)) / SUM(f.sale_dollars)) * 100, 2) AS total_gross_margin_percent

FROM
  sales.sales_gold.fact_sales AS f
JOIN
  sales.sales_gold.dim_seller AS s ON f.idSeller = s.idSeller
GROUP BY
  s.vendor_no
ORDER BY
  total_gross_margin_dollars DESC;

En esta consulta se visualiza la rentabilidad de cada vendedor, identificado por su número de vendedor.
La consulta calcula los ingresos totales, el costo total y el margen de beneficio bruto total para cada vendedor, mostrando el resultado tanto en dólares como en porcentaje.
Los resultados están ordenados por el margen en dólares en forma descendente, mostrando los vendedores más rentables primero.

## Optimización de precios

### Relación entre el precio minorista y el volumen de venta

Para contestar esta pregunta se analizará la elasticidad de precios.
En caso de aumentar un precio, que impacto tiene en el volumen de venta.

In [0]:
%sql
WITH PricePointAnalysis AS (
  -- Calculo el volumen y los ingresos para cada producto EN CADA precio
  SELECT
    p.itemno,
    p.itemName,
    f.state_bottle_retail AS price_point,
    SUM(f.sale_bottles) AS total_bottles_sold,
    ROUND(SUM(f.sale_dollars), 2) AS total_revenue
  FROM
    sales.sales_gold.fact_sales AS f
  JOIN
    sales.sales_gold.dim_product AS p ON f.idProduct = p.idProduct
  GROUP BY
    p.itemno,
    p.itemName,
    f.state_bottle_retail
),

RankedPricePoints AS (
  SELECT
    *,
    -- Contar cuántos puntos de precio (filas) hay por cada 'itemno'
    COUNT(*) OVER (PARTITION BY itemno) AS price_point_count
  FROM
    PricePointAnalysis
)

-- Seleccionamos solo los productos que tienen más de un precio
SELECT
  itemno,
  itemName,
  price_point,
  total_bottles_sold,
  total_revenue
FROM
  RankedPricePoints
WHERE
  price_point_count > 1
  --and itemno = 266 --En caso de analizar un producto en específico introducir el itemno
ORDER BY
  itemno,
  itemName,
  price_point;

En esta consulta se analiza la elasticidad de precios de los productos.
Se visualiza el itemno de un producto, los distintos precios a los que se ha vendido, el total de botellas vendidas a ese precio, y el ingreso total generado en ese mismo punto de precio.
La consulta filtra automáticamente y muestra únicamente aquellos productos que se han vendido a más de un precio distinto. Esto es fundamental para el análisis, ya que permite comparar cómo reacciona el volumen de ventas cuando cambia el precio de un mismo artículo.
Los resultados se ordenan por itemno y price_point, facilitando la comparación directa del rendimiento de un producto a sus diferentes precios. La consulta también incluye una línea comentada donde se puede introducir un itemno específico para aislar el análisis a un solo producto.
Los productos que no se encuentren en la lista no podrán ser sujetos a este análisis debido a que solo fueron vendidos a un único precio.

Por ejemplo se puede analizar que el producto de itemno = 227 es un producto elastico porque sube el precio y disminuye el volumen de ventas.
Por otro lado el producto de itemno = 352 es un producto inelastico porque sube el precio y el volumen de ventas aumento.

A aquellos productos inelásticos se puede llevar a cabo una revisión para aumentar el precio, sin embargo, aquellos productos elásticos se debe valorar mantener o disminuir el precio.